In [1]:
## Import Libraries
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import random
import torch
import seaborn as sns
from collections import Counter
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential,layers, losses, optimizers
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from keras.layers import GaussianNoise
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score

In [2]:
## Loading the CICIDS 2017 dataset
df_test1 = pd.read_csv("all_group_train_normalized.csv")

In [3]:
## Test benign data
df_test = df_test1[df_test1['Label'] == 'Benign']

In [4]:
df_test = df_test.iloc[:, 0:200]

In [5]:
# df_test_ben = df_test[df_test['Label'] == 'Benign'].iloc[:,0:200]

In [6]:
## Loading benign data from CICIDS 2017 dataset
df1 = pd.read_pickle("fwd_ben_pkt_data_wednesday_total.pkl")

In [7]:
## Normalizing the values
benign_data = df1.iloc[:,0:200]/255

In [8]:
benign_data

,column0,column1,column2,column3,column4,column5,column6,column7,column8,column9,...,column190,column191,column192,column193,column194,column195,column196,column197,column198,column199
0,0.003922,0.239216,0.000000,0.623529,0.25098,0.0,0.501961,0.462745,0.254902,0.231373,...,0.729412,0.886275,0.776471,0.890196,0.384314,0.956863,0.039216,0.854902,0.513725,0.203922
1,0.019608,0.784314,0.000000,0.627451,0.25098,0.0,0.501961,0.443137,0.709804,0.231373,...,0.815686,0.101961,0.035294,0.270588,0.356863,0.223529,0.729412,0.305882,0.066667,0.592157
2,0.031373,0.741176,0.000000,0.631373,0.25098,0.0,0.501961,0.431373,0.749020,0.231373,...,0.250980,0.858824,0.360784,0.741176,0.737255,0.333333,0.137255,0.964706,0.141176,0.529412
3,0.000000,0.156863,0.000000,0.639216,0.25098,0.0,0.501961,0.466667,0.321569,0.231373,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.003922,0.768627,0.054902,0.152941,0.25098,0.0,0.250980,0.584314,0.678431,0.886275,...,0.380392,0.729412,0.447059,0.470588,0.517647,0.764706,0.015686,0.698039,0.301961,0.482353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,0.000000,0.156863,0.207843,0.360784,0.25098,0.0,0.501961,0.913725,0.125490,0.858824,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
999996,0.000000,0.156863,0.207843,0.364706,0.25098,0.0,0.501961,0.913725,0.121569,0.858824,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
999997,0.000000,0.156863,0.207843,0.368627,0.25098,0.0,0.501961,0.913725,0.117647,0.858824,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
999998,0.000000,0.156863,0.490196,0.635294,0.25098,0.0,0.501961,0.627451,0.850980,0.356863,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [9]:
from keras.layers import LeakyReLU

In [14]:
## Architecture of the baseline model
tf.random.set_seed(2)
input_dims = 200
model = Sequential()
model.add(Dense(input_dims, input_shape = (input_dims, )))
model.add(Dense(150))
model.add(LeakyReLU(alpha=0.05))
model.add(Dense(150))
model.add(LeakyReLU(alpha=0.05))
model.add(Dense(input_dims // 2, activation='relu'))
model.add(Dense(input_dims // 4, activation='relu'))
model.add(Dense(input_dims // 8, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(6, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(input_dims // 8, activation='relu'))
model.add(Dense(input_dims // 4, activation='relu'))
model.add(Dense(input_dims // 2))
model.add(Dense(150))
model.add(LeakyReLU(alpha=0.05))
model.add(Dense(150))
model.add(LeakyReLU(alpha=0.05))
model.add(Dense(input_dims, activation='relu'))

In [17]:
## Compilation and fitting
model.compile(optimizer='adam', loss='mae')
model.fit(benign_data , benign_data , batch_size=256, epochs=500, shuffle=True)

Epoch 1/500
3907/3907 [==============================] - 27s 6ms/step - loss: 0.0156
Epoch 2/500
3907/3907 [==============================] - 25s 6ms/step - loss: 0.0135
Epoch 3/500
3907/3907 [==============================] - 25s 6ms/step - loss: 0.0126
Epoch 4/500
3907/3907 [==============================] - 25s 6ms/step - loss: 0.0123
Epoch 5/500
3907/3907 [==============================] - 25s 6ms/step - loss: 0.0120
Epoch 6/500
3907/3907 [==============================] - 25s 6ms/step - loss: 0.0119
Epoch 7/500
3907/3907 [==============================] - 28s 7ms/step - loss: 0.0118
Epoch 8/500
3907/3907 [==============================] - 44s 11ms/step - loss: 0.0117
Epoch 9/500
3907/3907 [==============================] - 44s 11ms/step - loss: 0.0115
Epoch 10/500
3907/3907 [==============================] - 44s 11ms/step - loss: 0.0114
Epoch 11/500
3907/3907 [==============================] - 45s 11ms/step - loss: 0.0114
Epoch 12/500
3907/3907 [==============================] - 3

3907/3907 [==============================] - 38s 10ms/step - loss: 0.0080
Epoch 191/500
3907/3907 [==============================] - 38s 10ms/step - loss: 0.0080
Epoch 192/500
3907/3907 [==============================] - 38s 10ms/step - loss: 0.0080
Epoch 193/500
3907/3907 [==============================] - 38s 10ms/step - loss: 0.0080
Epoch 194/500
3907/3907 [==============================] - 38s 10ms/step - loss: 0.0080
Epoch 195/500
3907/3907 [==============================] - 38s 10ms/step - loss: 0.0080
Epoch 196/500
3907/3907 [==============================] - 37s 10ms/step - loss: 0.0080
Epoch 197/500
3907/3907 [==============================] - 38s 10ms/step - loss: 0.0080
Epoch 198/500
3907/3907 [==============================] - 39s 10ms/step - loss: 0.0080
Epoch 199/500
3907/3907 [==============================] - 39s 10ms/step - loss: 0.0081
Epoch 200/500
3907/3907 [==============================] - 38s 10ms/step - loss: 0.0080
Epoch 201/500
3907/3907 [=====================

KeyboardInterrupt: 

In [78]:
## Saving the baseline autoencoder model
model.save('Stage2_AE.h5')

In [11]:
## Loading the autoencoder baseline model
from tensorflow.keras.models import load_model

model = load_model('Stage2_AE.h5', compile = False)

In [18]:
### Threshold Construction

In [12]:
## Pbre metric
def recon_metrices(data, reconstructed_data):

    maes = np.absolute(data.values - reconstructed_data)
    
    for j in range(len(maes)):
        m = maes[j]
        d = data.values[j]
        r = reconstructed_data[j]
        for el in range(len(m)):
            if r[el] == 0 or d[el] == 0:
                m[el] = 0
 
            
    mae_byte_list = []
    ip_mae_list = []
    tcp_header_mae_list = []
    tcp_options_mae_list = []
    tcp_segment_mae_list = []
    i=0
    for mae in maes:
#         print(np.count_nonzero(reconstructed_data[i]))
#         print(np.count_nonzero(data.values[i]))
#         print(np.max(np.count_nonzero(reconstructed_data[i]),np.count_nonzero(data.values[i])))
        mae_byte = sum(mae) / np.count_nonzero(mae)
        mae_byte_list.append(mae_byte)
        i += 1
    return mae_byte_list

In [13]:
df_test.columns = df1.columns[0:200]

In [14]:
recon_ben = model.predict(df_test)

In [15]:
mae_byte_list = recon_metrices(df_test,recon_ben)

In [16]:
## Thresholds on 99,95,90 percentile of benign data
mae_byte_avg = sum(mae_byte_list) / len(mae_byte_list)
print('mae_byte_avg', mae_byte_avg)
mae_byte_per_95 = np.percentile(mae_byte_list, 99)
print('mae_byte_99_percentile', mae_byte_per_95)
mae_byte_per_95 = np.percentile(mae_byte_list, 95)
print('mae_byte_95_percentile', mae_byte_per_95)
mae_byte_per_95 = np.percentile(mae_byte_list, 90)
print('mae_byte_90_percentile', mae_byte_per_95)

mae_byte_avg 0.05939387636491722
mae_byte_99_percentile 0.18711863867387962
mae_byte_95_percentile 0.089366675814845
mae_byte_90_percentile 0.07748561690180517


In [17]:
## Thresholds on 98 percentile of benign data
mae_byte_avg = sum(mae_byte_list) / len(mae_byte_list)
print('mae_byte_avg', mae_byte_avg)
mae_byte_per_95 = np.percentile(mae_byte_list, 98)

mae_byte_avg 0.05939387636491722
mae_byte_99_percentile 0.1341200515545619
mae_byte_95_percentile 0.089366675814845
mae_byte_90_percentile 0.07748561690180517


In [ ]:
## CICIDS2017 Testing

In [16]:
test_ben = benign_data.sample(frac = 0.2, random_state = 2).reset_index(drop = True)

In [17]:
recon_ben = model.predict(test_ben)

6250/6250 [==============================] - 15s 2ms/step


In [18]:
mae_byte_list = recon_metrices(test_ben,recon_ben)

In [18]:
## Classification
count_99 = len([i for i in mae_byte_list if i > 0.089])
print(count_99 / len(mae_byte_list))

0.05086839347207666


In [ ]:
## Testing the Baseline autoencoder model against attack benign data from CICIDS 2018 and CICIDS 2017

In [2]:
infiltration_attack = df_test1[df_test1['Label'] == 'Infiltration'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(infiltration_attack.shape)
portscan_attack = df_test1[df_test1['Label'] == 'Port Scan'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(portscan_attack.shape)
ddos_attack = df_test1[df_test1['Label'] == 'DDoS'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(ddos_attack.shape)
bruteforce_attack = df_test1[df_test1['Label'] == 'Brute Force'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(bruteforce_attack.shape)
dos_attack = df_test1[df_test1['Label'] == 'DoS'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(dos_attack.shape)
web_attack = df_test1[df_test1['Label'] == 'Web Attack'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(web_attack.shape)

In [21]:
inf_ = model.predict(infiltration_attack)
port_ = model.predict(portscan_attack)
dos_ = model.predict(dos_attack)
ddos_ = model.predict(ddos_attack)
brute_ = model.predict(bruteforce_attack)
web_ = model.predict(web_attack)

In [34]:
mae_byte_list_inf  = recon_metrices(web_attack,web_)

In [35]:
## Classification
count_99 = len([i for i in mae_byte_list_inf if i > 0.09])
print(count_99 / len(mae_byte_list_inf))

0.9993262968785088


In [ ]:
## CICIDS 2018 Testing

In [30]:
df_test_2018 = pd.read_csv("all_group_train_normalized_2018.csv")

In [31]:
df_test_2018

,ip_header_byte_2,ip_header_byte_3,ip_header_byte_4,ip_header_byte_5,ip_header_byte_6,ip_header_byte_7,ip_header_byte_8,ip_header_byte_10,ip_header_byte_11,tcp_header_byte_4,...,tcp_segment_data_byte_1452,tcp_segment_data_byte_1453,tcp_segment_data_byte_1454,tcp_segment_data_byte_1455,tcp_segment_data_byte_1456,tcp_segment_data_byte_1457,tcp_segment_data_byte_1458,tcp_segment_data_byte_1459,Label,Label_binary
0,0.000000,0.156863,0.498039,0.968627,0.25098,0.0,0.501961,0.619608,0.517647,0.356863,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign,Benign
1,0.000000,0.203922,0.309804,0.274510,0.25098,0.0,0.498039,0.760784,0.478431,0.490196,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ddos,Malicious
2,0.003922,0.678431,0.156863,0.870588,0.25098,0.0,0.247059,0.298039,0.270588,0.380392,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DoS,Malicious
3,0.000000,0.203922,0.078431,0.929412,0.25098,0.0,0.250980,0.901961,0.392157,0.541176,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign,Benign
4,0.000000,0.203922,0.125490,0.811765,0.25098,0.0,0.247059,0.333333,0.803922,0.298039,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DoS,Malicious
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291153,0.000000,0.156863,0.196078,0.360784,0.25098,0.0,0.501961,0.925490,0.125490,0.858824,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign,Benign
291154,0.000000,0.156863,0.152941,0.027451,0.25098,0.0,0.501961,0.968627,0.458824,0.858824,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign,Benign
291155,0.000000,0.235294,0.850980,0.992157,0.25098,0.0,0.247059,0.898039,0.168627,0.709804,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DoS,Malicious
291156,0.000000,0.156863,0.219608,0.054902,0.25098,0.0,0.501961,0.901961,0.427451,0.356863,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign,Benign


In [32]:
infiltration_attack = df_test_2018[df_test_2018['Label'] == 'Infiltration'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(infiltration_attack.shape)
portscan_attack = df_test_2018[df_test_2018['Label'] == 'Port Scan'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(portscan_attack.shape)
ddos_attack = df_test_2018[df_test_2018['Label'] == 'ddos'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(ddos_attack.shape)
bruteforce_attack = df_test_2018[df_test_2018['Label'] == 'Brute Force'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(bruteforce_attack.shape)
dos_attack = df_test_2018[df_test_2018['Label'] == 'DoS'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(dos_attack.shape)
web_attack = df_test_2018[df_test_2018['Label'] == 'Web Attack'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(web_attack.shape)
benign_2018 = df_test_2018[df_test_2018['Label'] == 'Benign'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(benign_2018.shape)

(29116, 200)
(0, 200)
(29115, 200)
(29116, 200)
(29116, 200)
(29116, 200)
(145579, 200)


In [33]:
inf_ = model.predict(infiltration_attack)
dos_ = model.predict(dos_attack)
ddos_ = model.predict(ddos_attack)
brute_ = model.predict(bruteforce_attack)
web_ = model.predict(web_attack)
benign_ = model.predict(benign_2018)

4550/4550 [==============================] - 11s 2ms/step


In [38]:
mae_byte_list_inf  = recon_metrices(web_attack,web_)

In [39]:
## Classification
count_99 = len([i for i in mae_byte_list_inf if i > 0.09])
print(count_99 / len(mae_byte_list_inf))

0.9687113614507488


In [24]:
adv_all = pd.read_csv("adv_examples_all_allDNN_bigger.csv")

In [46]:
adv_random = pd.read_csv("Adv_all_DNN_truerandom.csv")

In [54]:
adv_random = adv_random.loc[:, ~adv_random.columns.str.contains('^Unnamed')]

In [55]:
adv_ = adv_random.iloc[:,0:200]/256

In [56]:
adv_

,ip_header_byte_2,ip_header_byte_3,ip_header_byte_4,ip_header_byte_5,ip_header_byte_6,ip_header_byte_7,ip_header_byte_8,ip_header_byte_10,ip_header_byte_11,tcp_header_byte_4,...,tcp_segment_data_byte_125,tcp_segment_data_byte_126,tcp_segment_data_byte_127,tcp_segment_data_byte_128,tcp_segment_data_byte_129,tcp_segment_data_byte_130,tcp_segment_data_byte_131,tcp_segment_data_byte_132,tcp_segment_data_byte_133,tcp_segment_data_byte_134
0,0.000031,0.002681,0.000858,0.003676,0.00098,0.0,0.001961,0.001976,0.002696,0.000766,...,0.000597,0.001685,0.001670,0.001486,0.001716,0.000597,0.000490,0.001608,0.001762,0.000490
1,0.000000,0.000919,0.002665,0.001379,0.00098,0.0,0.000950,0.000352,0.001808,0.000153,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000031,0.000429,0.001593,0.003508,0.00098,0.0,0.001961,0.001256,0.001195,0.000751,...,0.000597,0.001685,0.001670,0.001486,0.001716,0.000597,0.000490,0.001608,0.001762,0.000490
3,0.000046,0.000337,0.000567,0.002160,0.00098,0.0,0.001961,0.002267,0.002635,0.000766,...,0.000597,0.001685,0.001670,0.001486,0.001716,0.000597,0.000490,0.001608,0.001762,0.000490
4,0.000015,0.002742,0.003493,0.001241,0.00098,0.0,0.000950,0.003431,0.000107,0.000751,...,0.000735,0.000735,0.000751,0.000735,0.000751,0.000490,0.001072,0.001608,0.001746,0.001547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9402,0.000061,0.000230,0.001945,0.002129,0.00098,0.0,0.001961,0.000873,0.002773,0.000766,...,0.001685,0.001670,0.001486,0.001716,0.000597,0.000490,0.001608,0.001762,0.000490,0.001685
9403,0.000015,0.002436,0.001670,0.002129,0.00098,0.0,0.001961,0.001195,0.000567,0.000766,...,0.000597,0.001685,0.001670,0.001486,0.001716,0.000597,0.000490,0.001608,0.001762,0.000490
9404,0.000000,0.000797,0.002773,0.002267,0.00098,0.0,0.000950,0.000245,0.001042,0.001563,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9405,0.000031,0.001915,0.000934,0.000169,0.00098,0.0,0.000950,0.002053,0.002022,0.000781,...,0.001700,0.001685,0.001762,0.001700,0.001654,0.001547,0.000705,0.001654,0.001700,0.001578


In [57]:
recon_adv = model.predict(adv_)

294/294 [==============================] - 1s 2ms/step


In [58]:
mae_byte_list_adv = recon_metrices(adv_,recon_adv)

In [59]:
mae_byte_avg = sum(mae_byte_list_adv) / len(mae_byte_list_adv)
print('mae_byte_avg', mae_byte_avg)
mae_byte_per_95 = np.percentile(mae_byte_list_adv, 99)
print('mae_byte_99_percentile', mae_byte_per_95)
mae_byte_per_95 = np.percentile(mae_byte_list_adv, 95)
print('mae_byte_95_percentile', mae_byte_per_95)
mae_byte_per_95 = np.percentile(mae_byte_list_adv, 90)
print('mae_byte_90_percentile', mae_byte_per_95)

mae_byte_avg 0.33242928722290327
mae_byte_99_percentile 0.3465080766152046
mae_byte_95_percentile 0.33478620288041583
mae_byte_90_percentile 0.3340043695358669


In [61]:
## Classification
count_99 = len([i for i in mae_byte_list_adv if i > 0.09])
print(count_99 / len(mae_byte_list_adv))

1.0


In [39]:
infiltration_attack = df_test1[df_test1['Label'] == 'Infiltration'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(infiltration_attack.shape)
portscan_attack = df_test1[df_test1['Label'] == 'Port Scan'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(portscan_attack.shape)
ddos_attack = df_test1[df_test1['Label'] == 'DDoS'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(ddos_attack.shape)
bruteforce_attack = df_test1[df_test1['Label'] == 'Brute Force'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(bruteforce_attack.shape)
dos_attack = df_test1[df_test1['Label'] == 'DoS'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(dos_attack.shape)
web_attack = df_test1[df_test1['Label'] == 'Web Attack'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(web_attack.shape)

(8905, 200)
(8905, 200)
(8905, 200)
(8905, 200)
(8906, 200)
(8906, 200)


In [40]:
recon_inf = model.predict(infiltration_attack)
recon_port = model.predict(portscan_attack)
recon_dos = model.predict(dos_attack)
recon_ddos = model.predict(ddos_attack)

279/279 [==============================] - 1s 2ms/step


In [41]:
mae_byte_list_inf = recon_metrices(infiltration_attack,recon_inf)
mae_byte_list_port = recon_metrices(portscan_attack,recon_port)
mae_byte_list_dos = recon_metrices(dos_attack,recon_dos)
mae_byte_list_ddos = recon_metrices(ddos_attack,recon_ddos)

In [61]:
## Classification
count_99 = len([i for i in mae_byte_list_inf if i > 0.13])
print(count_99 / len(mae_byte_list_inf))

0.6381807973048849


In [43]:
## Classification
count_99 = len([i for i in mae_byte_list_port if i > 0.09])
print(count_99 / len(mae_byte_list_port))

0.9998877035373386


In [44]:
## Classification
count_99 = len([i for i in mae_byte_list_dos if i > 0.09])
print(count_99 / len(mae_byte_list_dos))

0.9991017291713452


In [45]:
## Classification
count_99 = len([i for i in mae_byte_list_ddos if i > 0.09])
print(count_99 / len(mae_byte_list_ddos))

0.9359910162829871
